In [ ]:
def align_and_merge_dataframes(df_dict, fill_method='bfill'):
    """
    Aligns and merges multiple daily DataFrames into a single DataFrame without NaN values.
    Replace NaN with next/previous value in the beginning/end 

    Parameters:
    - df_dict (dict): A dictionary where keys are the desired column names in the merged DataFrame,
                      and values are pandas DataFrames with a DateTimeIndex.
    - fill_method (str): Method to fill NaN values. Options include 'bfill', 'ffill', 'interpolate'.

    Returns:
    - pd.DataFrame: The merged DataFrame containing all aligned data without NaN values.
    """
    if not df_dict:
        raise ValueError("The input dictionary of DataFrames is empty.")

    # Ensure all DataFrames have a DateTimeIndex
    for name, df in df_dict.items():
        if not isinstance(df.index, pd.DatetimeIndex):
            raise TypeError(f"The DataFrame for '{name}' does not have a DateTimeIndex.")

    # 1. Determine Global Start and End Dates
    start_date = min(df.index.min() for df in df_dict.values())
    end_date = max(df.index.max() for df in df_dict.values())

    # 2. Create Master Daily Date Range
    master_freq = 'D'
    master_dates = pd.date_range(start=start_date, end=end_date, freq=master_freq)

    # 3. Reindex and Fill Each DataFrame
    filled_dfs = {}
    for name, df in df_dict.items():
        # Reindex to master_dates
        reindexed_df = df.reindex(master_dates)

        # Fill NaNs based on the specified method
        if fill_method == 'bfill':
            filled_df = reindexed_df.bfill()
        elif fill_method == 'ffill':
            filled_df = reindexed_df.ffill()
        elif fill_method == 'interpolate':
            filled_df = reindexed_df.interpolate(method='linear')
        else:
            raise ValueError("fill_method must be 'bfill', 'ffill', or 'interpolate'")

        # After filling, handle remaining NaNs (if any)
        filled_df = filled_df.fillna(method='ffill').fillna(method='bfill')

        filled_dfs[name] = filled_df

    # 4. Merge All DataFrames
    merged_df = pd.concat(filled_dfs.values(), axis=1)
    merged_df.columns = filled_dfs.keys()

    # 5. Verify No NaN Values
    nan_counts = merged_df.isnull().sum()
    if nan_counts.sum() == 0:
        print("Merged DataFrame:")
        print(merged_df)
        print("\nMerged DataFrame successfully created with no NaN values.")
    else:
        print("There are still NaN values in the merged DataFrame:")
        print(merged_df[merged_df.isnull().any(axis=1)])

    return merged_df